In [126]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2


# if cannot import the modules, add the parent directory to system path might help

import os
import sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/'
sys.path.append(parent_dir)


import os
from utils.path import dir_HugeFiles
from utils.preprocessing_0716 import load
from utils.words import make_corpus_0, get_wordcount_list
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv, print_time
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import inflect
from collections import Counter

#dir_save = os.path.normpath(dir_HugeFiles+'dph/dic_20190607.pickle')
#dic = load(dir_save)
random_seed = 2019

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10 µs
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 29 ms


In [2]:
dic = load(dir_save = dir_HugeFiles+'nlg/dic_20190716.pickle')
ls = [i for i,v in dic.items() if len(v['ingredients'])>1]
print('drop %d recipes with less than 2 ingredients' %(len(dic)-len(ls)))
ls = [i for i in ls if len(dic[i]['directions'])>1]
print('furthur drop %d recipes with less than 2 instructions' %(len(dic)-len(ls)))
desc = [i for i in ls if len(dic[i]['description'])<1]
print('drop %d recipes with no description' %(len(desc)))
print('now we are using recipe54k %d' % len(ls))

exist
drop 46 recipes with less than 2 ingredients
furthur drop 1026 recipes with less than 2 instructions
drop 0 recipes with no description
now we are using recipe54k 54076
time: 2.22 s


In [98]:
df = pd.read_csv(dir_HugeFiles+'raw_data/food_taxonomy.txt',delimiter='\t', header = None)

time: 27.4 ms


In [69]:
ls = range(100)

time: 16.9 ms


In [99]:
def cleaning(df):
    '''
    eliminate the row if it contains the following non-ingredient words
    '''
    print('origin: length of data: %d' % len(df))
    eliminate = ['Snack brand', 'Preparation', 'Fast food', 'Dietary Supplement', 'Dessert']
    for i in range(2):
        df  = df[df.apply(lambda x: x[i] not in eliminate, axis = 1)]
    # uncased
    for i in range(3):
        df[i]  = df.apply(lambda x: x[i].lower(), axis = 1)
    print('drop some fields: length of data: %d' % len(df))    
    
    # drop the duplicates
    df = df[~df[2].duplicated()]
    df = df.reset_index(drop = True)
    print('drop duplicates: length of data: %d' % len(df))
        
    return df
df = cleaning(df)

origin: length of data: 4294
drop some fields: length of data: 2682
drop duplicates: length of data: 2582
time: 959 ms


In [6]:
for i in range(3):
    print(df[i].unique())

['bean and legume' 'beverage' 'breakfast cereal' 'condiment'
 'egg and dairy' 'fruit' 'herb and spice' 'meat' 'mushroom' 'nut and seed'
 'side dish' 'snack' 'staple' 'vegetable']
['legume' 'soy product' 'alcohol' 'chocolate' 'coffee' 'energy drink'
 'fruit juice' 'hot beverage' 'lemon beverage' 'milk substitutes'
 'rice beverage' 'soft drink' 'breakfast cereal' 'condiment'
 'dairy product' 'egg' 'berry' 'mediterranean' 'temperate' 'tropical'
 'herbs' 'mixtures' 'peppers' 'spices' 'beef' 'fish' 'game' 'lamb' 'meat'
 'meatball' 'pork' 'poultry' 'sausage' 'seafood' 'shellfish' 'mushroom'
 'gymnosperm seed' 'nut' 'other' 'pseudocereal' 'antipasto' 'fries'
 'legumes' 'maize' 'pasta' 'potatoes' 'salad' 'soup' 'snack' 'banana'
 'other cereal' 'rice' 'root and tuber' 'wheat' 'bulb and stem vegetables'
 'flowers and flower buds' 'fruits' 'leafy and salad' 'podded vegetables'
 'root and tuberous vegetables' 'sea vegetables' 'vegetables']
['bean' 'chickpea' 'cowpea' ... 'vegetables' 'vegetarian' 

In [171]:
p = inflect.engine()
class closest:
    def __init__(self, series):
        find_ngram = series.apply(lambda x: ' ' in x)
        self.unigram_ingredients, self.ngram_ingredients = series[~find_ngram], series[find_ngram]
        self.unigram_ingredients_pluraled = [p.plural(ingr) for ingr in self.unigram_ingredients]
    
    def ngram(self, listofline, pluraled_listofline):
        answer = []
        for ngram in self.ngram_ingredients.values:
            n_list= ngram.split(' ')
            intersection = set(listofline) & set(n_list)
            if len(intersection) >= 2:
                answer.append(ngram)
            else:
                # repeat
                intersection = (set(pluraled_listofline) | set(intersection)) & set(n_list)
                if len(intersection) >= 2:
                    answer.append(ngram)
        return set(answer)
    
    def unigram(self, listofline, pluraled_listofline):
        first =  set(listofline) & set(self.unigram_ingredients)
        if first:
            return first
        second = set(pluraled_listofline) & set(self.unigram_ingredients)
        if second:
            return second
    def check(self, listofline):
        pluraled_listofline = [p.plural(word) for word in listofline]
        ngram = self.ngram(listofline, pluraled_listofline)
        if ngram:
            return ngram
        unigram = self.unigram(listofline, pluraled_listofline)
        if unigram:
            return unigram

time: 20.9 ms


stage1: matching ingredients with food-taxonomy (from Aek) and known_ingr from Salvador

In [172]:
salvador = load_pickle(dir_HugeFiles+'raw_data/recipe1M/ingr_vocab.pkl')
known_ingr = pd.concat([df[2], pd.Series([ingr.replace('_',' ') for ingr in salvador if '<' not in ingr])], ignore_index=True)
known_ingr = known_ingr[~known_ingr.duplicated()]
model = closest(known_ingr)

time: 171 ms


In [ ]:
# may takes ~40 minutes
transformeds = []
for i, v in dic.items():
    if i in ls:
        ingredients = [sent.split(' ') for sent in v['ingredients']]
        transformed = [model.check(ingred) for ingred in ingredients]
        dic[i]['ingred_level2'] = transformed
        transformeds += transformed
    if not i % 1000:
        print(i)

save_pickle(obj=dic, filename=dir_HugeFiles+'dic_20190721.pickle')

Find the popular ingredients (ocurrence > 100) in salvador's list. <br>
Maunally add them to the food-taxonomy

In [24]:
# take 1 min
dic = load_pickle(filename=dir_HugeFiles+'dic_20190721.pickle')
transformeds = []
for i, v in dic.items():
    if i in ls:
        transformed = [ing for ingr in v['ingred_level2'] if ingr for ing in ingr]
        transformeds += transformed

time: 27.4 s


In [34]:
# count the occurrences of ingr indentified by salvador
count =  dict(Counter(transformeds))
common_sal = [k for k, v in count.items() if v>100 and k not in df[2].values]

time: 160 ms


In [173]:
additional_ingr=\
[
['Condiment', 'Sweet', 'sugar'],
['Condiment', 'Sweet', 'applesauce'],
['Flour', 'Flour','flour'],
['Baking powder','Baking powder','baking powder'],
['Water','Water','water'],
['Water','Water','iced'],
['Water','Water','ice'],
['Herb and spice','spices','jalapeno'],
['Condiment','Condiment', 'oil'],
['Beverage','Fruit juice', 'juice'],
['Staple','Maize','enchilada'],
['Condiment','Condiment', 'dressing mix'],
['Condiment','Condiment', 'cheese mix'],
['Baking powder','Baking powder','baking soda'],
['Condiment','Condiment', 'dry mix'],
['Condiment','Condiment','chocolate'],
['Vegetable','Bulb and stem vegetables','tapioca'],
['Flour', 'Flour','xanthan gum'],
['Flour', 'Flour','starch'],
['Egg and dairy', 'Dairy product','buttermilk'],
['Condiment','Condiment','chili'],
['Condiment','Condiment','chile'],
['Condiment','Condiment','chilis'],
['Condiment','Condiment','chiles'],
['Condiment','Condiment','chilies'],
['Flour', 'Flour','corn muffin mix'],
['Beverage','Chocolate','chocolate mix'],
['Meat','dumpling','dumpling'],
['Meat','dumpling','wonton'],
['Staple','Wheat','pizza dough'],
['Staple','Wheat','dough'],
['Condiment','Condiment', 'pizza sauce'],
['Flour', 'Flour','yeast'],
['Condiment','Sweet','cocoa'],
['Staple','Maize','chip'],
['Egg and dairy','Dairy product','ricotta'],
['Condiment','Condiment','seasoning'],
['Beverage','Alcohol','sherry'],
['Staple','Rice','grain rice'], 
['Staple','Wheat','shell'],
['Meat','Beef','fillet'],
['Staple','Maize','cornmeal'],
['Condiment','Condiment','seed oil'],
['Nut and seed','Other','seed'],
['Condiment', 'Sweet', 'sugar blend'],
['Soup','Soup','broth'],
['Soup','Soup','stock'],
['Condiment', 'Sweet', 'marshmallow'],
['Condiment', 'Dry Condiment', 'dried vegetable flakes'],
['Condiment', 'Dry Condiment', 'dried celery flakes'],
['Flour', 'Flour','cornstarch'],
['Staple','Wheat','double crust'],
['Staple','Wheat','crust'],
['Staple','Wheat','pastry crust'],
['Egg and dairy','Dairy product','gorgonzola'],
['Beverage','juice','drink mix'],
['Egg and dairy','Egg','Egg whites'],
['Baking powder','Baking powder','baking mix'],
['Staple','Rice','brown rice'],
['Condiment','Condiment','five spice'],
['Meat','Beef','tenderloin'],
['Meat','Pork','prosciutto'],
['Condiment', 'Sweet', 'whipped topping'],
['Condiment', 'Sweet', 'topping'],
['Beverage','Alcohol','cider'],
['Meat','Shellfish','crabmeat'],
['Condiment', 'Sweet', 'candy'],
['Condiment', 'Sweet', 'caramel'],
['Condiment', 'Sweet', 'molasses'],
['Vegetable','Podded vegetables','cannellini'],
['Vegetable','Fruits','fruit'],
['Staple','Wheat','saltine'],
['Condiment','Condiment', 'habanero'],
['Beverage','Juice','jell o'],
['Beverage','Juice','jelly'],
['Beverage','Soft drink','carbonated beverage'],
['Egg and dairy','Dairy product','gruyere'],
['Vegetable','Leafy and Salad','beet'],
['Water','Water','icing'],
['Egg and dairy','Dairy product','parmigiano'],
['Beverage','Alcohol','liqueur'],
['Condiment','Condiment', 'lard'],
['Staple','Wheat','crumb'],
['Herb and spice','Herb','peppermint'],
['Beverage','Alcohol','marsala'],
['Side dish','Potatoes','hash brown'],
['Meat','Beef','steak'],
['Condiment','Condiment','gelatin'],
['Meat','Beef','chuck'],
['Egg and dairy','Dairy product','colby'],
['Condiment', 'Sweet', 'jam'],
['Condiment', 'Sweet', 'cool whip'],
['Condiment', 'Sweet', 'stevia'],
['Staple','Wheat','bran'],
['Condiment','Condiment','pimento'],
['Condiment','Condiment','food coloring'],
['Meat','Meat','rib'],
['Condiment','Condiment','shortening'],
['Vegetable','Fruits','sweet pickles'],
['Condiment', 'Sweet', 'white confectioner'],
['Condiment', 'Sweet', 'confectioner'],  
['Vegetable','Root and tuberous vegetabless','rhubarb'],
['Condiment', 'Condiment', 'cooking spray']
]

def add_rows(df, additional_ingr):
    add = pd.DataFrame(additional_ingr)
    print('before processing: length of data: %d' % len(df))
    df = pd.concat([df,add]).reset_index(drop =True)

    # uncased
    for i in range(3):
        df[i]  = df.apply(lambda x: x[i].lower().strip(), axis = 1)
    print('add some rows: length of data: %d' % len(df))    
    
    # drop the duplicates
    df = df[~df[2].duplicated()]
    df = df.reset_index(drop = True)
    print('drop duplicates: length of data: %d' % len(df))
    return df

df = add_rows(df, additional_ingr)

before processing: length of data: 2684
add some rows: length of data: 2787
drop duplicates: length of data: 2684
time: 549 ms


In [174]:
# prepare the ingredient detection algorithm
model = closest(df[2])

# may takes ~24 minutes
transformeds = []
for i, v in dic.items():
    if i in ls:
        ingredients = [sent.split(' ') for sent in v['ingredients']]
        transformed = [model.check(ingred) for ingred in ingredients]
save_pickle(obj=dic, filename=dir_HugeFiles+'dic_20190722.pickle', overwrite = False)

already exists/data/yueliu/RecipeAnalytics_201906/dic_20190722.pickle
time: 25min 13s


In [177]:
# take 1 min
#dic = load_pickle(filename=dir_HugeFiles+'dic_20190722.pickle')

def conversion_rate(dic):
    converts, transformeds = [], []
    for i, v in dic.items():
        if i in ls:
            convert = len([ingr for ingr in v['ingred_level2'] if not ingr])
            converts.append(convert)
            transformeds += v['ingred_level2']
    converts = pd.Series(converts)
    print('%.4f recipes was almost converted' % (sum(converts<=1)/len(converts)))
    print('%.4f recipes was fully converted' % (sum(converts==0)/len(converts)))
    
    print('%d lines of ingredient in total' % len(transformeds))
    print('%.4f lines of ingredient was transformed' % (len([t for t in transformeds if t])/len(transformeds)))
    return converts, transformeds

converts, transformeds = conversion_rate(dic)

0.9801 recipes was almost converted
0.8597 recipes was fully converted
512222 lines of ingredient in total
0.9826 lines of ingredient was transformed
time: 20.3 s


In [179]:
# count the occurrences of ingr indentified by salvador
count =  dict(Counter([t for tr in transformeds if tr for t in tr]))
rare = [k for k, v in count.items() if v < 5]
rare

['pasta salad',
 'cheeses',
 '7up',
 'sub',
 'arame',
 'chile con queso',
 'lovage',
 'minute maid',
 'brinjal',
 'sugars',
 'jujube',
 'swiss miss',
 'delmonico steak',
 'elephant garlic',
 'loquats',
 'ibarra',
 'maize',
 'flap steak',
 'nacho',
 'puffed rice',
 'gelato',
 'ostrich',
 'sangria',
 'mochi',
 'fish paste',
 'marlin',
 'leches',
 'maitake mushroom',
 'raisin',
 'banana ketchup',
 'celtuce',
 'mutton',
 'blueberry jam',
 'jasmine tea',
 'herbal tea',
 'capellini',
 'tia maria',
 'ti',
 'sandwiches',
 'pomelo',
 'shichimi',
 'panch phoron',
 'lotus root',
 'lapsang souchong',
 'pomegranates',
 'mojito',
 'clam sauce',
 'poke',
 'boysenberry',
 'white tea',
 'horned melon',
 'curry leaf tree',
 'curry leaf',
 'chips & salsa',
 'abalone',
 'shark',
 'rice bran oil',
 'pheasants',
 'chive',
 'remoulade',
 'pepsi cola',
 'sassafras',
 'soybean',
 'blade steak',
 'risotto',
 'guavas',
 'special k',
 'grouse',
 'royal crown cherry',
 'royal crown cola',
 'chimichurri',
 'pollock

time: 159 ms
